In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-basis-constructor

# 分数ゲートの入門

*使用量の目安: Heron r2 プロセッサで30秒未満（注意: これは目安です。実際の実行時間は異なる場合があります。）*
## 背景
### IBM QPU における分数ゲート
分数ゲートは、パラメータ化された量子ゲートであり、任意の角度の回転（特定の範囲内）を直接実行することを可能にします。
これにより、複数の基底ゲートへの分解が不要になります。
物理量子ビット間のネイティブな相互作用を活用することで、ハードウェア上でより効率的に特定のユニタリ演算を実装できます。

IBM Quantum&reg; Heron QPU は以下の分数ゲートをサポートしています:
- $R_{ZZ}(\theta)$ for $0 < \theta < \pi / 2$
- $R_X(\theta)$ for any real value $\theta$

これらのゲートは、量子回路の深さと実行時間の両方を大幅に削減できます。
特に $R_{ZZ}$ と $R_X$ を多用するアプリケーションにおいて有利です。
例えば、ハミルトニアンシミュレーション、量子近似最適化アルゴリズム（QAOA）、量子カーネル法などが挙げられます。
本チュートリアルでは、実践的な例として量子カーネルに焦点を当てます。
### 制限事項
分数ゲートは現在実験的な機能であり、いくつかの制約があります:
- $R_{ZZ}$ の角度は $0 < \theta < \pi / 2$ の範囲に制限されています。
- 分数ゲートの使用は、[動的回路](/guide/classical-feedforward-and-control-flow)、[パウリ・ツイリング](/guides/error-mitigation-and-suppression-techniques#pauli-twirling)、[確率的エラーキャンセレーション](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-cancellation-pec)（PEC）、および[ゼロノイズ外挿](/guides/error-mitigation-and-suppression-techniques#zero-noise-extrapolation-zne)（ZNE）（[確率的エラー増幅](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-amplification-pea)（PEA）を使用する場合）ではサポートされていません。

分数ゲートは、標準的なアプローチとは異なるワークフローを必要とします。
本チュートリアルでは、実践的なアプリケーションを通じて分数ゲートの使い方を説明します。

分数ゲートの詳細については、以下を参照してください。
- [分数ゲート](/guides/fractional-gates)
- [分数ゲートを使用*しない*方がよい場合](/guides/fractional-gates#when-not-to-use)
## 概要
分数ゲートを使用するワークフローは、一般的に [Qiskit パターン](/guides/intro-to-patterns)のワークフローに従います。
主な違いは、すべての RZZ 角度が $0 < \theta \leq \pi/2$ の制約を満たす必要があることです。
この条件を確保するためのアプローチは2つあります。
本チュートリアルでは、2番目のアプローチに焦点を当て、推奨します。

### 1. RZZ 角度制約を満たすパラメータ値を生成する
すべての RZZ 角度が有効範囲内にあることが確実な場合、標準的な Qiskit パターンのワークフローに従うことができます。
この場合、パラメータ値を PUB の一部として送信するだけです。ワークフローは以下のように進みます。

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import unitary_overlap
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

有効範囲外の角度を持つ RZZ ゲートを含む PUB を送信しようとすると、以下のようなエラーメッセージが表示されます:
```
'The instruction rzz is supported only for angles in the range [0, pi/2], but an angle (20.0) outside of this range has been requested; via parameter value(s) γ[0]=10.0, substituted in parameter expression 2.0*γ[0].'
```
このエラーを回避するには、以下に説明する2番目のアプローチを検討してください。
### 2. トランスパイル前にパラメータ値を回路に割り当てる
`qiskit-ibm-runtime` パッケージは、[`FoldRzzAngle`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/transpiler-passes-fold-rzz-angle) という専用のトランスパイラパスを提供しています。
このパスは、すべての RZZ 角度が RZZ 角度制約に準拠するように量子回路を変換します。
`generate_preset_pass_manager` または `transpile` にバックエンドを指定すると、Qiskit は自動的に `FoldRzzAngle` を量子回路に適用します。
この方法では、トランスパイル前にパラメータ値を量子回路に割り当てる必要があります。
ワークフローは以下のように進みます。

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=133
)  # backend should be a heron device or later
backend_name = backend.name
backend_c = service.backend(backend_name)  # w/o fractional gates
backend_f = service.backend(
    backend_name, use_fractional_gates=True
)  # w/ fractional gates
print(f"Backend: {backend_name}")
print(f"No fractional gates: {backend_c.basis_gates}")
print(f"With fractional gates: {backend_f.basis_gates}")
if "rzz" not in backend_f.basis_gates:
    print(f"Backend {backend_name} does not support fractional gates")

Backend: ibm_fez
No fractional gates: ['cz', 'id', 'rz', 'sx', 'x']
With fractional gates: ['cz', 'id', 'rx', 'rz', 'rzz', 'sx', 'x']


このワークフローは、パラメータ値を量子回路に割り当て、パラメータが束縛された回路をローカルに保存する必要があるため、最初のアプローチよりも計算コストが高くなることに注意してください。
また、Qiskit には特定のシナリオで RZZ ゲートの変換が失敗する既知の問題があります。回避策については、[トラブルシューティング](#troubleshooting)セクションを参照してください。
本チュートリアルでは、量子カーネル法に着想を得た例を通じて、2番目のアプローチで分数ゲートを使用する方法を説明します。
量子カーネルが有用である可能性が高い場面をより深く理解するために、[Liu, Arunachalam & Temme (2021)](https://www.nature.com/articles/s41567-021-01287-z) を読むことをお勧めします。

また、[量子カーネルトレーニング](/tutorials/quantum-kernel-training)チュートリアルや、IBM Quantum Learning の量子機械学習コースにおける[量子カーネル](/learning/courses/quantum-machine-learning/quantum-kernel-methods)レッスンも併せてご参照ください。
### 前提条件
本チュートリアルを始める前に、以下がインストールされていることを確認してください:
- Qiskit SDK v2.0 以降、[可視化](https://docs.quantum.ibm.com/api/qiskit/visualization)サポート付き
- Qiskit Runtime v0.37 以降（`pip install qiskit-ibm-runtime`）
- Qiskit Basis Constructor（`pip install qiskit_basis_constructor`）
### セットアップ

In [3]:
optimization_level = 2
shots = 2000
reps = 3
rng = np.random.default_rng(seed=123)

In [4]:
def my_zz_feature_map(num_qubits: int, reps: int = 1) -> QuantumCircuit:
    x = ParameterVector("x", num_qubits * reps)
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))
    for k in range(reps):
        K = k * num_qubits
        for i in range(num_qubits):
            qc.rz(x[i + K], i)
        pairs = [(i, i + 1) for i in range(num_qubits - 1)]
        for i, j in pairs[0::2] + pairs[1::2]:
            qc.rzz((np.pi - x[i + K]) * (np.pi - x[j + K]), i, j)
    return qc


def quantum_kernel(num_qubits: int, reps: int = 1) -> QuantumCircuit:
    qc = my_zz_feature_map(num_qubits, reps=reps)
    inner_product = unitary_overlap(qc, qc, "x", "y", insert_barrier=True)
    inner_product.measure_all()
    return inner_product


def random_parameters(inner_product: QuantumCircuit) -> np.ndarray:
    return np.tile(rng.random(inner_product.num_parameters // 2), 2)


def fidelity(result) -> float:
    ba = result.data.meas
    return ba.get_int_counts().get(0, 0) / ba.num_shots

Quantum kernel circuits and their corresponding parameter values are generated for systems with 4 to 40 qubits, and their fidelities are subsequently evaluated.

In [5]:
qubits = list(range(4, 44, 4))
circuits = [quantum_kernel(i, reps=reps) for i in qubits]
params = [random_parameters(circ) for circ in circuits]

## 分数ゲートを用いたワークフロー
### ステップ 1: 古典的な入力を量子問題にマッピングする
#### 量子カーネル回路
このセクションでは、RZZ ゲートを使用した量子カーネル回路を探究し、分数ゲートのワークフローを紹介します。

まず、カーネル行列の個々の要素を計算するための量子回路を構築します。
これは、ZZ 特徴マップ回路とユニタリオーバーラップを組み合わせることで行います。
カーネル関数は、特徴マップ空間のベクトルを受け取り、カーネル行列の要素として内積を返します:
$$K(x, y) = \langle \Phi(x) | \Phi(y) \rangle,$$
ここで $|\Phi(x)\rangle$ は特徴マップされた量子状態を表します。

RZZ ゲートを使用して、ZZ 特徴マップ回路を手動で構築します。
Qiskit には組み込みの `zz_feature_map` が提供されていますが、Qiskit v2.0.2 の時点では RZZ ゲートをサポートしていません（[issue を参照](https://github.com/Qiskit/qiskit/issues/14469)）。

次に、同一の入力に対するカーネル関数を計算します。例えば $K(x, x) = 1$ となります。
ノイズのある量子コンピュータでは、ノイズの影響によりこの値が1未満になる場合があります。
1に近い結果ほど、実行時のノイズが少ないことを示します。
本チュートリアルでは、この値を*忠実度*と呼び、以下のように定義します。
$$\text{fidelity} = K(x, x).$$

In [6]:
circuits[0].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/b3d6341a-0.avif" alt="Output of the previous code cell" />

In the standard Qiskit patterns workflow, parameter values are typically passed to the Sampler or Estimator primitive as part of a PUB.
However, when using a backend that supports fractional gates, these parameter values must be explicitly assigned to the quantum circuit prior to transpilation.

In [7]:
b_qc = [
    circ.assign_parameters(param) for circ, param in zip(circuits, params)
]
b_qc[0].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/6c9c1977-0.avif" alt="Output of the previous code cell" />

4量子ビットから40量子ビットまでのシステムに対して、量子カーネル回路とそれに対応するパラメータ値が生成され、その後忠実度が評価されます。

In [8]:
backend_f = service.backend(name=backend_name, use_fractional_gates=True)
# pm_f includes `FoldRzzAngle` pass
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_f
)

In [9]:
t_qc_f = pm_f.run(b_qc)
print(t_qc_f[0].count_ops())
t_qc_f[0].draw("mpl", fold=-1)

OrderedDict([('rz', 35), ('rzz', 18), ('x', 13), ('rx', 9), ('measure', 4), ('barrier', 2)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/a18e5c70-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/b3d6341a-0.avif)

標準的な Qiskit パターンのワークフローでは、パラメータ値は通常 PUB の一部として Sampler または Estimator プリミティブに渡されます。
しかし、分数ゲートをサポートするバックエンドを使用する場合、これらのパラメータ値はトランスパイル前に量子回路に明示的に割り当てる必要があります。

In [10]:
nnl_f = [qc.num_nonlocal_gates() for qc in t_qc_f]
depth_f = [qc.depth() for qc in t_qc_f]
duration_f = [
    qc.estimate_duration(backend_f.target, unit="u") for qc in t_qc_f
]

![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/6c9c1977-0.avif)

### ステップ 2: 量子ハードウェア実行のための問題の最適化

次に、標準的な Qiskit パターンに従い、パスマネージャーを使用して回路をトランスパイルします。
分数ゲートをサポートするバックエンドを `generate_preset_pass_manager` に提供することで、`FoldRzzAngle` という専用パスが自動的に含まれます。
このパスは、RZZ 角度制約に準拠するように回路を変更します。
その結果、前の図で負の値を持っていた RZZ ゲートは正の値に変換され、いくつかの追加の X ゲートが挿入されます。

In [11]:
sampler_f = SamplerV2(mode=backend_f)
sampler_f.options.dynamical_decoupling.enable = True
sampler_f.options.dynamical_decoupling.sequence_type = "XY4"
sampler_f.options.dynamical_decoupling.skip_reset_qubits = True

In [12]:
job = sampler_f.run(t_qc_f, shots=shots)
print(job.job_id())

d4bninsi51bc738j97eg


### Step 4: Post-process and return result in desired classical format

You can obtain the kernel function value $K(x, x)$ by measuring the probability of the all-zero bitstring `00...00` in the output.

In [13]:
# job = service.job("d1obougt0npc73flhiag")
result = job.result()
fidelity_f = [fidelity(result=res) for res in result]
print(fidelity_f)
usage_f = job.usage()

[0.9005, 0.647, 0.3345, 0.355, 0.3315, 0.174, 0.1875, 0.149, 0.1175, 0.085]


![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/a18e5c70-1.avif)

分数ゲートの効果を評価するために、非局所ゲートの数（このバックエンドでは CZ と RZZ）、
回路の深さと実行時間を評価し、後ほど標準ワークフローのものと比較します。

In [14]:
# step 1: map classical inputs to quantum problem
# `circuits` and `params` from the previous section are reused here

In [15]:
# step 2: optimize circuits
backend_c = service.backend(backend_name)  # w/o fractional gates
pm_c = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_c
)
t_qc_c = pm_c.run(circuits)
print(t_qc_c[0].count_ops())
t_qc_c[0].draw("mpl", fold=-1)

OrderedDict([('rz', 130), ('sx', 80), ('cz', 36), ('measure', 4), ('barrier', 2)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/a10f2d95-1.avif" alt="Output of the previous code cell" />

In [16]:
nnl_c = [qc.num_nonlocal_gates() for qc in t_qc_c]
depth_c = [qc.depth() for qc in t_qc_c]
duration_c = [
    qc.estimate_duration(backend_c.target, unit="u") for qc in t_qc_c
]

In [17]:
# step 3: execute
sampler_c = SamplerV2(backend_c)
sampler_c.options.dynamical_decoupling.enable = True
sampler_c.options.dynamical_decoupling.sequence_type = "XY4"
sampler_c.options.dynamical_decoupling.skip_reset_qubits = True

In [18]:
job = sampler_c.run(pubs=zip(t_qc_c, params), shots=shots)
print(job.job_id())

d4bnirvnmdfs73ae3a2g


In [19]:
# step 4: post-processing
# job = service.job("d1obp8j3rr0s73bg4810")
result = job.result()
fidelity_c = [fidelity(res) for res in result]
print(fidelity_c)
usage_c = job.usage()

[0.6675, 0.5725, 0.098, 0.102, 0.065, 0.0235, 0.006, 0.0015, 0.0015, 0.002]


## 分数ゲートなしのワークフローと回路の比較
このセクションでは、分数ゲートをサポートしないバックエンドを使用した標準的な Qiskit パターンのワークフローを示します。
トランスパイルされた回路を比較すると、分数ゲートを使用したバージョン（前のセクション）の方が、分数ゲートなしのバージョンよりもコンパクトであることがわかります。

In [20]:
plt.plot(qubits, depth_c, "-o", label="no fractional gates")
plt.plot(qubits, depth_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("depth")
plt.title("Comparison of depths")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/ef343a53-1.avif" alt="Output of the previous code cell" />

In [21]:
plt.plot(qubits, duration_c, "-o", label="no fractional gates")
plt.plot(qubits, duration_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("duration (µs)")
plt.title("Comparison of durations")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/98bb2cd0-1.avif" alt="Output of the previous code cell" />

In [22]:
plt.plot(qubits, nnl_c, "-o", label="no fractional gates")
plt.plot(qubits, nnl_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("number of non-local gates")
plt.title("Comparison of numbers of non-local gates")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/1383b242-1.avif" alt="Output of the previous code cell" />

In [23]:
plt.plot(qubits, fidelity_c, "-o", label="no fractional gates")
plt.plot(qubits, fidelity_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("fidelity")
plt.title("Comparison of fidelities")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/8b4594f5-1.avif" alt="Output of the previous code cell" />

We compare the QPU usage time with and without fractional gates. The results in the following cell show that the QPU usage times are almost identical.

In [24]:
print(f"no fractional gates: {usage_c} seconds")
print(f"fractional gates: {usage_f} seconds")

no fractional gates: 7 seconds
fractional gates: 7 seconds


## Advanced topic: Using only fractional RX gates

The need for the modified workflow when using fractional gates primarily stems from the restriction on RZZ gate angles.
However, if you use only the fractional RX gates and exclude the fractional RZZ gates, you can continue to follow the standard Qiskit patterns workflow.
This approach can still offer meaningful benefits, particularly in circuits that involve a large number of RX gates and U gates, by reducing the overall gate count and potentially improving performance.
In this section, we demonstrate how to optimize your circuits using only fractional RX gates, while omitting RZZ gates.

To support this, we provide a utility function that allows you to disable a specific basis gate in a Target object.
Here, we use it to disable RZZ gates.

In [25]:
from qiskit.circuit.library import n_local
from qiskit.transpiler import Target

In [26]:
def remove_instruction_from_target(target: Target, gate_name: str) -> Target:
    new_target = Target(
        description=target.description,
        num_qubits=target.num_qubits,
        dt=target.dt,
        granularity=target.granularity,
        min_length=target.min_length,
        pulse_alignment=target.pulse_alignment,
        acquire_alignment=target.acquire_alignment,
        qubit_properties=target.qubit_properties,
        concurrent_measurements=target.concurrent_measurements,
    )

    for name, qarg_map in target.items():
        if name == gate_name:
            continue
        instruction = target.operation_from_name(name)
        if qarg_map == {None: None}:
            qarg_map = None
        new_target.add_instruction(instruction, qarg_map, name=name)
    return new_target

We use a circuit consisting of U, CZ, and RZZ gates as an example.

In [27]:
qc = n_local(3, "u", "cz", "linear", reps=1)
qc.rzz(1.1, 0, 1)
qc.draw("mpl")

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/6b812497-0.avif" alt="Output of the previous code cell" />

We first transpile the circuit for a backend that does not support fractional gates.

In [28]:
pm_c = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_c
)
t_qc = pm_c.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 23), ('sx', 16), ('cz', 4)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/9e8e0709-1.avif" alt="Output of the previous code cell" />

## 深さと忠実度の比較
このセクションでは、フラクショナルゲートを使用した回路と使用しない回路の間で、非局所ゲートの数と忠実度を比較します。
これにより、実行効率と品質の観点からフラクショナルゲートを使用する潜在的なメリットが明らかになります。

In [29]:
backend_f = service.backend(backend_name, use_fractional_gates=True)
target = remove_instruction_from_target(backend_f.target, "rzz")
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level,
    target=target,
)
t_qc = pm_f.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 22), ('sx', 14), ('cz', 4), ('rx', 1)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/db45feb0-1.avif" alt="Output of the previous code cell" />

### Optimize U gates with fractional RX gates

In this section, we demonstrate how to optimize U gates using fractional RX gates, building on the same circuit introduced in the previous section.

You will need to install the `qiskit-basis-constructor` [package](https://github.com/Qiskit/qiskit-basis-constructor) for this section.
This is a beta version of a new transpilation plugin for Qiskit, which might be integrated into Qiskit in the future.

In [30]:
# %pip install qiskit-basis-constructor

In [31]:
from qiskit.circuit.library import UGate
from qiskit_basis_constructor import DEFAULT_EQUIVALENCE_LIBRARY

We transpile the circuit using only fractional RX gates, excluding RZZ gates.
By introducing a custom decomposition rule, as shown in the following,
we can reduce the number of single-qubit gates required to implement a U gate.

This feature is currently under discussion in this [GitHub issue.](https://github.com/Qiskit/qiskit/issues/13455)

In [32]:
# special decomposition rule for UGate
x = ParameterVector("x", 3)
zxz = QuantumCircuit(1)
zxz.rz(x[2] - np.pi / 2, 0)
zxz.rx(x[0], 0)
zxz.rz(x[1] + np.pi / 2, 0)
DEFAULT_EQUIVALENCE_LIBRARY.add_equivalence(UGate(x[0], x[1], x[2]), zxz)

Next, we apply the transpiler using `constructor-beta` translation provided by `qiskit-basis-constructor` package.
As a result, the total number of gates is reduced compare to the previous transpilation.

In [33]:
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level,
    target=target,
    translation_method="constructor-beta",
)
t_qc = pm_f.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 16), ('rx', 9), ('cz', 4)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/b19aae7c-1.avif" alt="Output of the previous code cell" />

## Troubleshooting

### Issue: Invalid RZZ angles might remain after transpilation

As of Qiskit v2.0.3, there are known issues where RZZ gates with invalid angles may remain in the circuits even after transpilation.
The issue typically arises under the following conditions.

#### Failure when using `target` option with `generate_preset_pass_manager` or `transpiler`

When the `target` option is used with `generate_preset_pass_manager` or `transpiler`, the specialized transpiler pass `FoldRzzAngle` is not invoked.
To ensure proper handling of RZZ angles for fractional gates, we recommend always using the `backend` option instead.
See [this issue](https://github.com/Qiskit/qiskit/issues/14318) for more details.

#### Failure when circuits contain certain gates

If your circuit includes certain gates such as `XXPlusYYGate`, the Qiskit transpiler may generate RZZ gates with invalid angles.
If you encounter this issue, see this [GitHub issue](https://github.com/Qiskit/qiskit-ibm-runtime/issues/2256#issuecomment-2889487152) for a workaround.

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_cCNiGkGX5xZMzoG)